In [1]:
from sklearn import linear_model
from sklearn.linear_model import LassoCV

In [2]:
import pandas as pd

In [3]:
import numpy as np

In [4]:
data_folder = "../HomePriceBeastNew/"

In [6]:
combined_home_data = pd.read_csv(f"{data_folder}combined_home_data.csv",low_memory=True)

C:\Users\spsundar.FAREAST\Anaconda3\envs\TML\lib\site-packages\IPython\core\interactiveshell.py:3169: DtypeWarning: Columns (97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,169,245,284,288,292,304,308,312,348,352,356) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [7]:
def ProcessCombinedHomeData(combined_home_data):

    known_columns = {'county_name':'str','state_code':'str',
    'period_month_year':'date','period_begin':'date','period_end':'date'}

    print('Processing Columns')
    for col in combined_home_data.columns:
        print(f'\t{col}')
        if col not in known_columns:
            combined_home_data[col] = pd.to_numeric(combined_home_data[col],
                                                    errors='coerce')
            combined_home_data[col].fillna(0,inplace=True)

        elif known_columns[col] == 'date':
            combined_home_data[col] = pd.to_datetime(combined_home_data[col])
    
    return combined_home_data

In [8]:
def GetCensusColumns(df):
    lst = list(df.columns)
    
    prevIx = lst.index('FIPS_Code')
    
    colsOfInterest = lst[(prevIx+1):len(lst)]
    
    baseGame = {}
    for x in colsOfInterest:
        keyv = x.rsplit('_',1)
        baseGame[keyv[0]] = keyv[1]
    
    ret=[]
    for k in baseGame:
        ret.append(k+'_'+baseGame[k])
        
    return ret

In [9]:
def GetSortedFeatureDict(combined_home_data,yCol='median_sale_price'):
    censcols = GetCensusColumns(combined_home_data)
    censcols = [x for x in censcols if x!= yCol]
    X = combined_home_data[censcols].values
    y = combined_home_data[yCol].values

    reg = LassoCV(cv=20,
                  n_alphas=20,
                  random_state=0,
                  max_iter=2000,
                  normalize=True).fit(X, y)
    
    coefNZDict = {}
    for x in range(len(censcols)):
        if reg.coef_[x]!=0:
            coefNZDict[censcols[x]] = reg.coef_[x]
            
    return sorted(coefNZDict.items(), 
       key=lambda x: x[1],
       reverse=True)

In [10]:
combined_home_data = ProcessCombinedHomeData(combined_home_data)

Processing Columns
	region_id
	period_begin
	period_end
	total_homes_sold
	total_homes_sold_yoy
	average_homes_sold
	average_homes_sold_yoy
	total_homes_sold_with_price_drops
	total_homes_sold_with_price_drops_yoy
	average_homes_sold_with_price_drops
	average_homes_sold_with_price_drops_yoy
	percent_homes_sold_with_price_drops
	percent_homes_sold_with_price_drops_yoy
	median_sale_price
	median_sale_price_yoy
	median_sale_ppsf
	median_sale_ppsf_yoy
	median_days_to_close
	median_days_to_close_yoy
	price_drops
	price_drops_yoy
	percent_active_listings_with_price_drops
	percent_active_listings_with_price_drops_yoy
	pending_sales
	pending_sales_yoy
	median_pending_sqft
	median_pending_sqft_yoy
	off_market_in_two_weeks
	off_market_in_two_weeks_yoy
	off_market_in_one_week
	off_market_in_one_week_yoy
	percent_off_market_in_two_weeks
	percent_off_market_in_two_weeks_yoy
	percent_off_market_in_one_week
	percent_off_market_in_one_week_yoy
	total_new_listings
	total_new_listings_yoy
	average_new_l

In [11]:
combined_home_data['log_median_sale_price'] = np.log(combined_home_data['median_sale_price']+0.1 )

In [12]:
GetSortedFeatureDict(combined_home_data,yCol = 'log_median_sale_price')

[('R_INTERNATIONAL_MIG_2019', 0.17610948977884106),
 ('Unemployment_rate_2020', 0.12709234018235027),
 ('PCT_COLL_4_2015_19', 0.10089351079140449),
 ('PCT_COLL_1TO3_2000', 0.09006688977901284),
 ('PCT_HSD_Only_2000', 0.0879606476535308),
 ('R_NET_MIG_2019', 0.028895682161653686),
 ('Med_HH_Income_Percent_of_State_Total_2019', 0.023916752148838184),
 ('GQ_ESTIMATES_2019', 0.0009909080307563684),
 ('N_POP_CHG_2019', 0.0005525030638367441),
 ('INTERNATIONAL_MIG_2019', 0.00026240620112466404),
 ('NET_MIG_2019', 0.00024496245588270305),
 ('HSD_Only_2000', 0.00023712250784969354),
 ('DOMESTIC_MIG_2019', 0.00018635562239923184),
 ('RESIDUAL_2019', 2.6954919950351492e-05),
 ('Deaths_2019', -0.0002021831104990749),
 ('COLL_4_2000', -0.0002810094331158557),
 ('POP_ESTIMATE_2019', -0.0002872353305514814),
 ('LT_HSD_2015_19', -0.0004969433493877789),
 ('COLL_1TO3_2000', -0.0006302078882193471),
 ('Unemployed_2020', -0.0006335672359380096),
 ('NATURAL_INC_2019', -0.0007373928907110868),
 ('GQ_ESTIM